In [2]:
!pip install scikit-multilearn==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 5.2 MB/s eta 0:00:00


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.adapt import MLkNN
import sklearn.metrics as metrics
from sklearn.metrics import hamming_loss, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import multilabel_confusion_matrix
import seaborn as sns
import tensorflow as tf

In [2]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.optim import AdamW
import json
import nltk
import string
nltk.download('punkt')
wpt = nltk.WordPunctTokenizer()

d2= pd.read_csv('/content/drive/MyDrive/Nlp/uli_dataset-main/training/train_en_l1.csv')
d2
d2 = d2.rename(columns={'key' : 'unique_id', 'sentence' : 'text'})
d2.to_csv('updated_train_en_l1.csv', index=False)
# d2

# Convert annotator columns to numeric without replacing NaNs
d2[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']] = d2[
    ['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']
].apply(pd.to_numeric, errors='coerce')  # NaNs are retained

# Compute 'label' based on majority voting while ignoring NaNs
d2['label'] = (d2[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']].mean(axis=1, skipna=True) >= 0.5).astype(int)
d2



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,text,unique_id,en_a1,en_a2,en_a3,en_a4,en_a5,en_a6,label
0,"This is our history . For Shia and Sunni ,...",question_1,NaN,NaN,NaN,NaN,NaN,0.0,0
1,"If this is ""grilling"" then wht wud u call t...",question_1,NaN,NaN,NaN,1.0,NaN,NaN,1
2,look at <handle replaced>madam. Locals attac...,question_1,NaN,NaN,NaN,NaN,0.0,NaN,0
3,our beloved yogi ji has given money home and...,question_1,NaN,NaN,NaN,NaN,1.0,NaN,1
4,"""I gonna kill that bitch"" Bitvj imma kill you...",question_1,NaN,NaN,NaN,NaN,NaN,1.0,1
...,...,...,...,...,...,...,...,...,...
6526,“mama let’s research” bitch suck my dick 😭,question_1,NaN,NaN,NaN,NaN,0.0,NaN,0
6527,😂😂😂😂 I WAS BY MYSELF AND IT WAS HER AND HER BI...,question_1,NaN,NaN,NaN,NaN,0.0,NaN,0
6528,😂😂😂😂😂😂😂😂😂 i just seen a comment and bitch IM C...,question_1,NaN,NaN,NaN,NaN,0.0,NaN,0
6529,"😎""Boys Only Luxury!!!!!!!!!!!!!!!!!!!!!!!!!!!""...",question_1,0.0,1.0,1.0,0.0,NaN,NaN,1


In [3]:
# Create binary label ('hate' or 'not_hate')
def determine_binary_label(label):
    return 'hate' if label == 1 else 'not_hate'

d2['binary_label'] = d2['label'].apply(determine_binary_label)

# # Reorder columns
d2 = d2[['unique_id', 'text', 'binary_label', 'label']]

In [4]:
## Word Pre-Processing ##
import nltk
nltk.download('stopwords')
import string
import re
wpt = nltk.WordPunctTokenizer()
stop_words_init = nltk.corpus.stopwords.words('english')
stop_words = [i for i in stop_words_init if i not in ('not','and','for')]
print(stop_words)
## Function to normalize text for pre-processing ##
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'<.*?>+', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\w*\d\w*', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
## Apply the written function ##
d2.loc[:, 'text'] = d2['text'].apply(lambda x: normalize_text(x))
processed_list = []
for j in d2['text']:
    process = j.replace('...','')
    processed_list.append(process)

df_processed = pd.DataFrame(processed_list)
df_processed.columns = ['text']
df_processed.head(n=5)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she'd", "she'll",

,text
0,this is our history for shia and sunni don t l...
1,if this is grilling then wht wud u call th stu...
2,look at madam locals attacked no religion angl...
3,our beloved yogi ji has given money home and f...
4,i gonna kill that bitch bitvj imma kill you we...


In [5]:
X = list(df_processed['text'])
y = d2[['label']].values
y

array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, Activation, Dropout, Dense, Flatten,
    Bidirectional, GRU, concatenate, SpatialDropout1D,
    GlobalMaxPooling1D, GlobalAveragePooling1D, Conv1D,
    Embedding, Input, Concatenate
)
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import MeanSquaredError

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

######## Textual Features for Embedding ###################
max_len = 100
max_features = 4479

# Tokenization
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding
X = pad_sequences(X, padding='post', maxlen=max_len)

print(X)

[[  12    6   68 ...    0    0    0]
 [  34   12    6 ...    0    0    0]
 [ 158   64  239 ...    0    0    0]
 ...
 [  10   41  387 ...    0    0    0]
 [1590 1170   69 ...    0    0    0]
 [ 461  294   27 ...    0    0    0]]


In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 1, 0, ..., 0, 1, 0])

In [8]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=2)
y

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [9]:
with open('/content/drive/MyDrive/Nlp/uli_dataset-main/glove_embeddings.json', encoding="utf8") as f:
    embeddings_list = json.load(f)

# Convert the list of vectors to a dictionary with word indices as keys
embeddings_dictionary = {str(i): vector for i, vector in enumerate(embeddings_list)}

# Define tokenizer
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size
word_index = tokenizer.word_index
num_words = min(max_features, vocab_size)  # Limit vocab to max_features

# Get embedding dimension (from first vector in list)
embed_size = len(embeddings_list[0]) if embeddings_list else 0

# Initialize embedding matrix
embedding_matrix = np.zeros((num_words, embed_size))

# Fill embedding matrix with corresponding word vectors
for word, index in word_index.items():
    if index >= max_features:
        continue
    embedding_vector = embeddings_dictionary.get(word) or embeddings_dictionary.get(str(index))
    if embedding_vector is not None:
        embedding_matrix[index] = np.asarray(embedding_vector, dtype=np.float32)

print("Embedding matrix shape:", embedding_matrix.shape)

Embedding matrix shape: (4479, 50)


In [10]:
CNN_MODEL_DIR = '/content/drive/MyDrive/Nlp/models_1_2_en'
CNN_PLOTS_DIR = '/content/drive/MyDrive/Nlp/plots_cnn_en'
CNN_EVAL_DIR = '/content/drive/MyDrive/Nlp/eval_cnn_en'

In [11]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Input, Embedding, SpatialDropout1D, Conv1D,
    Bidirectional, LSTM, Dense, Dropout,
    GlobalAveragePooling1D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix

def create_cnn_bilstm_model(max_len, max_features, embedding_matrix, embed_size=300):
    """
    Creates the CNN-BiLSTM model architecture as described in the paper
    """
    # Input layer
    input_layer = Input(shape=(max_len,))

    # Embedding layer with pretrained weights (GloVe/FastText)
    embedding_layer = Embedding(
        input_dim=max_features,
        output_dim=embed_size,
        weights=[embedding_matrix],
        input_length=max_len,
        trainable=False  # As per paper, embeddings are non-trainable
    )(input_layer)

    # Spatial Dropout to prevent overfitting (as mentioned in paper)
    spatial_dropout = SpatialDropout1D(0.2)(embedding_layer)
    # CNN Layer (as described in paper)
    conv_layer = Conv1D(
        filters=64,  # As per paper
        kernel_size=2,  # As per paper
        activation='tanh',
        padding='same'
    )(spatial_dropout)

    # Bidirectional LSTM Layer (as described in paper)
    bilstm_layer = Bidirectional(
        LSTM(
            units=128,  # As per paper
            return_sequences=True,
            dropout=0.1,  # As per paper
            recurrent_dropout=0.5  # As per paper
        )
    )(conv_layer)

    # Global Average Pooling (as per paper)
    gap_layer = GlobalAveragePooling1D()(bilstm_layer)

    # Dense layer (as per paper)
    dense_layer = Dense(128, activation='relu')(gap_layer)
    dropout_layer = Dropout(0.2)(dense_layer)  # Additional dropout as per paper

    # Output layer (use float32 for softmax for numerical stability)
    output_layer = Dense(2, activation='softmax', dtype='float32')(dropout_layer)

    # Create model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

In [12]:
class MacroF1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes = 2, name='macro_f1_score', **kwargs):
        super(MacroF1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.tp = self.add_weight(name='tp', initializer='zeros')
        self.fp = self.add_weight(name='fp', initializer='zeros')
        self.fn = self.add_weight(name='fn', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert probabilities to predicted class indices
        y_pred = tf.argmax(y_pred, axis=-1)

        # Convert one-hot encoded y_true to class indices if needed
        if len(y_true.shape) > 1 and y_true.shape[-1] > 1:
            y_true = tf.argmax(y_true, axis=-1)

        # Initialize confusion matrix
        conf_matrix = tf.math.confusion_matrix(
            y_true,
            y_pred,
            num_classes=self.num_classes,
            dtype=tf.float32
        )

        # Calculate TP, FP, FN for each class
        diag = tf.linalg.diag_part(conf_matrix)
        row_sum = tf.reduce_sum(conf_matrix, axis=1)
        col_sum = tf.reduce_sum(conf_matrix, axis=0)

        tp = diag
        fp = col_sum - diag
        fn = row_sum - diag

        # Update the state variables
        self.tp.assign_add(tf.reduce_sum(tp))
        self.fp.assign_add(tf.reduce_sum(fp))
        self.fn.assign_add(tf.reduce_sum(fn))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        # Calculate precision and recall
        precision = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        recall = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())

        # Calculate F1 score
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

        # Return macro F1 (average of per-class F1 scores)
        return f1

    def reset_states(self):
        self.tp.assign(0.)
        self.fp.assign(0.)
        self.fn.assign(0.)
        self.count.assign(0.)

In [13]:
def train_and_validate_model(model, X_train, y_train, X_val, y_val, batch_size=128, epochs=5, model_dir=CNN_MODEL_DIR):
    """
    Trains the CNN-BiLSTM model with early stopping and model checkpointing
    Returns the best model and training history
    """
    # Create directory for saving models if it doesn't exist
    os.makedirs(model_dir, exist_ok=True)

    # Callbacks
    early_stopping = EarlyStopping(
        monitor='macro_f1_score',
        patience=2,
        restore_best_weights=True,
        mode='max',
        verbose=1
    )

    model_checkpoint = ModelCheckpoint(
        os.path.join(model_dir,  'best_model_en_task2_m1.h5'),  # Save entire model
        monitor='macro_f1_score',
        mode='max',
        save_best_only=True,
        verbose=1
    )

    # Compile model with Adam optimizer (as per paper)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', MacroF1Score(num_classes=2)]
    )

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=batch_size,
        epochs=epochs,
        callbacks=[early_stopping, model_checkpoint],
        verbose=1
    )

    # Load the best model found during training
    best_model = load_model(os.path.join(model_dir, 'best_model_en_task2_m1.h5'),
                          custom_objects={'MacroF1Score': MacroF1Score})

    return history, best_model



In [14]:
def plot_training_history(history, plot_dir=CNN_PLOTS_DIR):
    """
    Plots training history (accuracy and loss curves)
    Saves plots to specified directory
    """
    os.makedirs(plot_dir, exist_ok=True)

    # Plot training history
    plt.figure(figsize=(12, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, 'training_history_en_task1_m1.png'))
    plt.close()

In [18]:
def evaluate_validation(model, X_val, y_val, plot_dir=CNN_EVAL_DIR):
    """
    Evaluates the model on validation data and saves metrics and plots
    """
    os.makedirs(plot_dir, exist_ok=True)

    # Predict probabilities
    y_pred_proba = model.predict(X_val, batch_size=64)

    # Convert to class labels
    y_pred = np.argmax(y_pred_proba, axis=1)
    y_true = np.argmax(y_val, axis=1)

    # Calculate metrics
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    weighted_f1 = f1_score(y_true, y_pred, average='weighted')
    macro_f1 = f1_score(y_true, y_pred, average='macro')


    # Classification report
    report = classification_report(y_true, y_pred,labels=[0, 1], target_names=['not_hate', 'hate'])

    # Confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred)

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not Hate', 'Hate'],
                yticklabels=['Not Hate', 'Hate'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix (Validation)')
    plt.savefig(os.path.join(plot_dir, 'confusion_matrix_val_en_task2_m1.png'))
    plt.close()

    return {
        'precision': precision,
        'recall': recall,
        'f1_score_weighted': weighted_f1,
        'f1_score_macro': macro_f1,
        'classification_report': report,
        'confusion_matrix': conf_matrix
    }

In [19]:
def logistic_regression_baseline(X_train_text, X_val_text, y_train_lr, y_val_lr, save_dir='/content/drive/MyDrive/Nlp/lr_baseline'):
    import pickle
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
    os.makedirs(save_dir, exist_ok=True)

    # Create TF-IDF features
    tfidf = TfidfVectorizer(max_features=5000)
    X_train_tfidf = tfidf.fit_transform(X_train_text)
    X_val_tfidf = tfidf.transform(X_val_text)

    # Train logistic regression
    logreg = LogisticRegression(max_iter=1000)
    logreg.fit(X_train_tfidf, y_train_lr.ravel())

    # Evaluate on validation set
    y_pred_lr = logreg.predict(X_val_tfidf)
    precision = precision_score(y_val_lr, y_pred_lr, average='weighted')
    recall = recall_score(y_val_lr, y_pred_lr, average='weighted')
    weighted_f1 = f1_score(y_val_lr, y_pred_lr, average='weighted')
    macro_f1 = f1_score(y_val_lr, y_pred_lr, average='macro')
    report = metrics.classification_report(y_val_lr, y_pred_lr, target_names=['not_hate', 'hate'])
    conf_matrix_lr = confusion_matrix(y_val_lr, y_pred_lr)

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix_lr, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not Hate', 'Hate'],
                yticklabels=['Not Hate', 'Hate'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix (Logistic Regression)')
    plt.savefig(os.path.join(save_dir, 'confusion_matrix_lr.png'))
    plt.close()

    # Save the LR model and TF-IDF vectorizer
    with open(os.path.join(save_dir, 'logistic_regression_model.pkl'), 'wb') as f:
        pickle.dump({'model': logreg, 'tfidf': tfidf}, f)

    print("\nLogistic Regression Baseline Evaluation (Validation):")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Weighted F1 Score: {weighted_f1:.4f}")
    print(f"Macro F1 Score: {macro_f1:.4f}")
    print("\nClassification Report:")
    print(report)

    return logreg, tfidf

def evaluate_logistic_regression_on_test(logreg, tfidf, save_dir='/content/drive/MyDrive/Nlp/lr_baseline_en_task2_m1'):
    # Load and preprocess test data
    d2_test = pd.read_csv('/content/drive/MyDrive/Nlp/uli_dataset-main/training/train_en_l1.csv', engine='python', on_bad_lines='skip')
    d2_test = d2_test.rename(columns={'key': 'label', 'sentence': 'text'})
    d2_test[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']] = d2_test[
        ['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']
    ].apply(pd.to_numeric, errors='coerce')  # NaNs are retained
    d2_test['label'] = (d2_test[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']].mean(axis=1, skipna=True) >= 0.5).astype(int)

    d2_test['binary_label'] = d2_test['label'].apply(determine_binary_label)
    d2_test = d2_test[['text', 'binary_label', 'label']]
    d2_test['text'] = d2_test['text'].apply(lambda x: normalize_text(x))
    processed_list_test = [j.replace('...', '') for j in d2_test['text']]
    df_processed_test = pd.DataFrame(processed_list_test, columns=['text'])
    X_test_text = list(df_processed_test['text'])
    y_test = d2_test[['label']].values

    X_test_tfidf = tfidf.transform(X_test_text)
    y_pred_test = logreg.predict(X_test_tfidf)
    precision = precision_score(y_test, y_pred_test, average='weighted')
    recall = recall_score(y_test, y_pred_test, average='weighted')
    weighted_f1 = f1_score(y_test, y_pred_test, average='weighted')
    macro_f1 = f1_score(y_test, y_pred_test, average='macro')
    report = metrics.classification_report(y_test, y_pred_test, target_names=['not_hate', 'hate'])
    conf_matrix_test = confusion_matrix(y_test, y_pred_test)

    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix_test, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not Hate', 'Hate'],
                yticklabels=['Not Hate', 'Hate'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix (Logistic Regression Test)')
    plt.savefig(os.path.join(save_dir, 'confusion_matrix_lr_test.png'))
    plt.close()

    print("\nLogistic Regression Baseline Evaluation (Test):")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Weighted F1 Score: {weighted_f1:.4f}")
    print(f"Macro F1 Score: {macro_f1:.4f}")
    print("\nClassification Report:")
    print(report)

In [20]:
if __name__ == "__main__":
    # ------------------ CNN-BiLSTM Baseline ------------------ #
    X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
    )
    print(f"Training samples (CNN): {len(X_train)}")
    print(f"Validation samples (CNN): {len(X_val)}")
    embed_size = embedding_matrix.shape[1]
    cnn_model = create_cnn_bilstm_model(max_len, max_features, embedding_matrix, embed_size)
    cnn_model.summary()
    history, trained_cnn_model = train_and_validate_model(
        cnn_model, X_train, y_train, X_val, y_val,
        batch_size=128,
        epochs=5,
        model_dir=CNN_MODEL_DIR
    )
    plot_training_history(history, plot_dir=CNN_PLOTS_DIR)
    val_results = evaluate_validation(trained_cnn_model, X_val, y_val, plot_dir=CNN_EVAL_DIR)

    print("\nCNN-BiLSTM Validation Results:")
    print(f"Precision: {val_results['precision']:.4f}")
    print(f"Recall: {val_results['recall']:.4f}")
    print(f"Weighted F1 Score: {val_results['f1_score_weighted']:.4f}")
    print(f"Macro F1 Score: {val_results['f1_score_macro']:.4f}")
    print("\nClassification Report:")
    print(val_results['classification_report'])

    d2_test = pd.read_csv('/content/drive/MyDrive/Nlp/uli_dataset-main/testing/test_en_l1.csv', engine='python', on_bad_lines='skip')
    d2_test = d2_test.rename(columns={'key': 'label', 'sentence': 'text'})
    d2_test['binary_label'] = d2_test['label'].apply(determine_binary_label)
    d2_test = d2_test[['text', 'binary_label', 'label']]
    d2_test['text'] = d2_test['text'].apply(lambda x: normalize_text(x))
    processed_list_test = [j.replace('...', '') for j in d2_test['text']]
    df_processed_test = pd.DataFrame(processed_list_test, columns=['text'])
    X_test_text_cnn = list(df_processed_test['text'])
    # Use the same tokenizer for CNN baseline
    X_test_seq = tokenizer.texts_to_sequences(X_test_text_cnn)
    X_test_padded = pad_sequences(X_test_seq, padding='post', maxlen=max_len)

    test_results = evaluate_validation(trained_cnn_model, X_test_padded, to_categorical(label_encoder.fit_transform(d2_test[['label']].values.ravel()), num_classes=2), plot_dir=CNN_EVAL_DIR)

    print("\nCNN-BiLSTM Test Results:")
    print(f"Precision: {test_results['precision']:.4f}")
    print(f"Recall: {test_results['recall']:.4f}")
    print(f"Weighted F1 Score: {test_results['f1_score_weighted']:.4f}")
    print(f"Macro F1 Score: {test_results['f1_score_macro']:.4f}")
    print("\nClassification Report:")
    print(test_results['classification_report'])

    # ------------------ Logistic Regression Baseline ------------------ #
    # Split the original cleaned text for LR baseline
    X_text = list(df_processed['text']) # df_processed contains the cleaned text data
    X_train_text, X_val_text, y_train_lr, y_val_lr = train_test_split(
        X_text, d2['label'], test_size=0.2, random_state=42
    )

    lr_model, tfidf_vectorizer = logistic_regression_baseline(X_train_text, X_val_text, y_train_lr, y_val_lr, save_dir='/content/drive/MyDrive/Nlp/lr_baseline_en_l1')

    # Evaluate LR baseline on test set
    evaluate_logistic_regression_on_test(lr_model, tfidf_vectorizer, save_dir='/content/drive/MyDrive/Nlp/lr_baseline_en_l1')

Training samples (CNN): 5224
Validation samples (CNN): 1307


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 100, 50)        │       223,950 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_1             │ (None, 100, 50)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 100, 64)        │         6,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 100, 256)       │       197,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 461,200 (1.76 MB)

...

41/41 ━━━━━━━━━━━━━━━━━━━━ 18s 450ms/step - accuracy: 0.7902 - loss: 0.5110 - macro_f1_score: 0.7902 - val_accuracy: 0.8141 - val_loss: 0.4801 - val_macro_f1_score: 0.8141
Epoch 5/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step - accuracy: 0.7827 - loss: 0.5283 - macro_f1_score: 0.7827
Epoch 5: macro_f1_score improved from 0.78599 to 0.79173, saving model to /content/drive/MyDrive/Nlp/models_1_2_en/best_model_en_task2_m1.h5


41/41 ━━━━━━━━━━━━━━━━━━━━ 18s 449ms/step - accuracy: 0.7829 - loss: 0.5279 - macro_f1_score: 0.7829 - val_accuracy: 0.8179 - val_loss: 0.4730 - val_macro_f1_score: 0.8179
Restoring model weights from the end of the best epoch: 5.


21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 110ms/step

CNN-BiLSTM Validation Results:
Precision: 0.8514
Recall: 0.8179
Weighted F1 Score: 0.7450
Macro F1 Score: 0.4986

Classification Report:
              precision    recall  f1-score   support

    not_hate       0.82      1.00      0.90      1056
        hate       1.00      0.05      0.10       251

    accuracy                           0.82      1307
   macro avg       0.91      0.53      0.50      1307
weighted avg       0.85      0.82      0.75      1307

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step

CNN-BiLSTM Test Results:
Precision: 1.0000
Recall: 0.9928
Weighted F1 Score: 0.9964
Macro F1 Score: 0.4982

Classification Report:
              precision    recall  f1-score   support

    not_hate       1.00      0.99      1.00      1107
        hate       0.00      0.00      0.00         0

    accuracy                           0.99      1107
   macro avg       0.50      0.50      0.50      1107
weighted avg       1.00      0.99      1.00      1

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.p


Logistic Regression Baseline Evaluation (Validation):
Precision: 0.8177
Recall: 0.8347
Weighted F1 Score: 0.7993
Macro F1 Score: 0.6293

Classification Report:
              precision    recall  f1-score   support

    not_hate       0.84      0.98      0.91      1056
        hate       0.71      0.24      0.35       251

    accuracy                           0.83      1307
   macro avg       0.78      0.61      0.63      1307
weighted avg       0.82      0.83      0.80      1307


Logistic Regression Baseline Evaluation (Test):
Precision: 0.8351
Recall: 0.8366
Weighted F1 Score: 0.8030
Macro F1 Score: 0.6624

Classification Report:
              precision    recall  f1-score   support

    not_hate       0.84      0.98      0.90      5159
        hate       0.83      0.28      0.42      1372

    accuracy                           0.84      6531
   macro avg       0.83      0.63      0.66      6531
weighted avg       0.84      0.84      0.80      6531



In [ ]:
# ##TESTING###
# d2= pd.read_csv('/content/drive/MyDrive/Nlp/uli_dataset-main/testing/test_en_l1.csv')
# d2
# d2 = d2.rename(columns={'key' : 'unique_id', 'sentence' : 'text'})
# d2.to_csv('updated_test_en_l1.csv', index=False)
# # d2

# # Convert annotator columns to numeric without replacing NaNs
# d2[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']] = d2[
#     ['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']
# ].apply(pd.to_numeric, errors='coerce')  # NaNs are retained

# # Compute 'label' based on majority voting while ignoring NaNs
# d2['label'] = (d2[['en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6']].mean(axis=1, skipna=True) >= 0.5).astype(int)
# d2

,text,unique_id,en_a1,en_a2,en_a3,en_a4,en_a5,en_a6,label
0,"""who's this tweet about"" bitch it's about you ...",question_1,NaN,0.0,NaN,NaN,0.0,1.0,0
1,#AlkaLamba should’ve used her slippers to slap...,question_1,0.0,0.0,NaN,0.0,NaN,NaN,0
2,#CoronaVillains #COVIDIDIOT #COVID2019 is a...,question_1,NaN,NaN,1.0,NaN,1.0,1.0,1
3,"(Acts 29:19994) LIKE the female ""governors"" of...",question_1,NaN,1.0,1.0,NaN,1.0,NaN,1
4,***Let us celebrate “secular” as a beautiful w...,question_1,NaN,0.0,1.0,0.0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
1102,you should be tired of not having any kind of ...,question_1,0.0,NaN,1.0,NaN,NaN,1.0,1
1103,“The limits of tyrants are prescribed by the e...,question_1,0.0,NaN,0.0,NaN,0.0,NaN,0
1104,♋️🦀 like wtf my symbol is 69 and I’m a crab an...,question_1,NaN,NaN,0.0,NaN,0.0,1.0,0
1105,"🇺🇸: marry a millionaire guy, divorce get half ...",question_1,NaN,NaN,1.0,NaN,1.0,1.0,1


In [ ]:
# d2.loc[:, 'binary_label'] = d2['label'].apply(determine_binary_label)

# # # Reorder columns
# d2 = d2[['unique_id', 'text', 'binary_label', 'label']]

# d2.to_csv('updated_test_en_l1.csv', index=False)

# d2.loc[:, 'text'] = d2['text'].apply(lambda x: normalize_text(x))
# processed_list = []
# for j in d2['text']:
#     process = j.replace('...','')
#     processed_list.append(process)

# df_processed = pd.DataFrame(processed_list)
# df_processed.columns = ['text']
# df_processed.head(n=5)

# X = list(df_processed['text'])
# y = d2[['label']].values

# X = tokenizer.texts_to_sequences(X)

# # Padding
# X = pad_sequences(X, padding='post', maxlen=max_len)

# y = label_encoder.fit_transform(y.ravel())

# y = to_categorical(y, num_classes=2)

In [ ]:
# test_results = evaluate_validation(trained_model, X, y)

# print(r"\Test Results:")
# print(f"Precision: {val_results['precision']:.4f}")
# print(f"Recall: {val_results['recall']:.4f}")
# print(f"weighted F1 Score: {val_results['f1_score_weighted']:.4f}")
# print(f"macro F1 Score: {val_results['f1_score_macro']:.4f}")
# print("\nClassification Report:")
# print(val_results['classification_report'])

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step
\Test Results:
Precision: 0.8390
Recall: 0.8179
weighted F1 Score: 0.7463
macro F1 Score: 0.5020

Classification Report:
              precision    recall  f1-score   support

    not_hate       0.82      1.00      0.90      1056
        hate       0.93      0.06      0.11       251

    accuracy                           0.82      1307
   macro avg       0.87      0.53      0.50      1307
weighted avg       0.84      0.82      0.75      1307



In [ ]:
# def main():
#     # device = "cuda" if torch.cuda.is_available() else "cpu"
#     # print("\nRunning 10-fold cross-validation on training data...")
#     # cv_precision, cv_recall, cv_f1 = run_cross_validation(train_dataset_full, num_folds=10, num_epochs=7, batch_size=64, lr=1e-3, device=device)
#     # print("\nTraining final model on full 80% training set...")
#     # final_model = train_full_model(train_dataset_full, num_epochs=7, batch_size=64, lr=1e-3, device=device)
#     # torch.save(final_model.state_dict(), "final_cnn_bilstm_multilate_english.pt")
#     # print("Final model saved as 'final_cnn_bilstm_multilate_english.pt'.")
#     # test_loader = DataLoader(test_dataset, batch_size=64)
#     # final_precision, final_recall, final_f1 = evaluate_model(final_model, test_loader, device=device)
#     # print(f"\nFinal Test Set Evaluation: Precision = {final_precision:.4f}, Recall = {final_recall:.4f}, F1 = {final_f1:.4f}")
#     # # -------------------------------
#     # Alternatively,run without cross-validation:
#     # Uncomment the following lines:
#     model = CNN_BiLSTM(vocab_size=vocab_size, embedding_dim=EMBEDDING_DIM, embedding_matrix=embedding_matrix, num_classes=2)
#     print("Starting training of the CNN-BiLSTM model...")
#     train_model(model, train_loader, test_loader, num_epochs=7, lr=1e-3)
#     model.load_state_dict(torch.load("best_cnn_bilstm_model.pt"))
#     precision, recall, f1 = evaluate_model(model, test_loader, device="cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Final Test Set Evaluation: Precision = {precision:.4f}, Recall = {recall:.4f}, F1 = {f1:.4f}")
#     torch.save(model.state_dict(), "final_cnn_bilstm_model.pt")
#     print("Model saved as 'final_cnn_bilstm_model.pt'.")
#     # -------------------------------

# if __name__ == "__main__":
#     main()